<a href="https://colab.research.google.com/github/jungeKatz/AItweetBot/blob/main/tweet_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time


In [4]:
with open('/content/drive/MyDrive/nlp/extract.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))

text length 4576053


In [5]:
print(text[:250])


painful to see the devastating impact of extreme weather in western  with the unprecedented loss of life &amp; destructio n of property. may the world unite in preventing such climatic upheavals by investing in regenerating the planet. – sg what happ


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


499 unique characters


In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)


In [9]:
ids = ids_from_chars(chars)
ids


<tf.RaggedTensor [[37, 38, 39, 40, 41, 42, 43], [60, 61, 62]]>

In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [11]:
chars = chars_from_ids(ids)
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids


<tf.Tensor: shape=(4576053,), dtype=int64, numpy=array([52, 37, 45, ..., 50,  2,  2])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


p
a
i
n
f
u
l
 
t
o


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)


In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'p' b'a' b'i' b'n' b'f' b'u' b'l' b' ' b't' b'o' b' ' b's' b'e' b'e'
 b' ' b't' b'h' b'e' b' ' b'd' b'e' b'v' b'a' b's' b't' b'a' b't' b'i'
 b'n' b'g' b' ' b'i' b'm' b'p' b'a' b'c' b't' b' ' b'o' b'f' b' ' b'e'
 b'x' b't' b'r' b'e' b'm' b'e' b' ' b'w' b'e' b'a' b't' b'h' b'e' b'r'
 b' ' b'i' b'n' b' ' b'w' b'e' b's' b't' b'e' b'r' b'n' b' ' b' ' b'w'
 b'i' b't' b'h' b' ' b't' b'h' b'e' b' ' b'u' b'n' b'p' b'r' b'e' b'c'
 b'e' b'd' b'e' b'n' b't' b'e' b'd' b' ' b'l' b'o' b's' b's' b' ' b'o'
 b'f' b' ' b'l'], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())


b'painful to see the devastating impact of extreme weather in western  with the unprecedented loss of l'
b'ife &amp; destructio n of property. may the world unite in preventing such climatic upheavals by inve'
b'sting in regenerating the planet. \xe2\x80\x93 sg what happens within us is never determined by what is happenin'
b'g around us. but unfortunately, we link it to what is happening around us.  serpents\nmost maligned cr'
b'eatures in all of divine creation.\nthese gentle, colorful beings to me have not only\nbeen dear but de'


In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


In [21]:
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)


In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'painful to see the devastating impact of extreme weather in western  with the unprecedented loss of '
Target: b'ainful to see the devastating impact of extreme weather in western  with the unprecedented loss of l'


In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset


<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x


In [27]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 500) # (batch_size, sequence_length, vocab_size)


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()


In [30]:
sampled_indices


array([478, 383, 424, 362, 240, 196, 432, 431, 171, 194, 490, 391,  66,
       100, 220, 465, 417, 379, 212,  20, 417, 164, 144, 326, 361, 116,
       472,  78,  49, 352, 353, 266, 175, 262, 346,  72, 158, 495, 402,
       176,  61,  91, 398,  63, 339, 152, 148, 196, 424, 394, 429, 235,
       245, 290,  41, 116, 162,  32, 336,  94, 428, 451, 223, 141,  44,
        63, 286, 443, 343, 169, 264,  82,  15, 193, 161,  73,  88, 314,
        83, 244, 253,  26, 352, 383, 143,  29, 327, 446, 481, 150, 122,
       473, 373, 420,  64, 136, 294,  25,  32,  71])

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())


Input:
 b'w of love except the name? love comes with a knife,  not some shy question. -rumi osho on mother ter'

Next Char Predictions:
 b'\xf0\x9f\xa4\xa6\xf0\x9f\x90\xae\xf0\x9f\x98\x85\xf0\x9f\x8c\xb7\xe0\xb2\xb2\xe0\xae\xbf\xf0\x9f\x98\x8d\xf0\x9f\x98\x8c\xe0\xae\x92\xe0\xae\xb9\xf0\x9f\xa5\xb2\xf0\x9f\x91\x8d\xc2\xa0\xdc\x93\xe0\xb2\x96\xf0\x9f\x99\x88\xf0\x9f\x97\xa3\xf0\x9f\x8f\xbb\xe0\xb2\x852\xf0\x9f\x97\xa3\xe0\xae\x87\xe0\xa4\xb8\xef\xb8\x8f\xf0\x9f\x8c\x9d\xe0\xa4\x97\xf0\x9f\xa4\x94\xc3\xb0m\xf0\x9f\x8c\x94\xf0\x9f\x8c\x95\xe2\x80\x8e\xe0\xae\x9a\xe2\x80\x8a\xf0\x9f\x87\xb7\xc3\xa0\xe0\xa5\xa4\xf0\x9f\xa7\x85\xf0\x9f\x92\xaa\xe0\xae\x9cy\xd1\xb5\xf0\x9f\x92\x97|\xf0\x9d\x9a\x91\xe0\xa5\x83\xe0\xa4\xbf\xe0\xae\xbf\xf0\x9f\x98\x85\xf0\x9f\x92\x8c\xf0\x9f\x98\x8a\xe0\xb2\xac\xe0\xb2\xb8\xe2\x95\x90e\xe0\xa4\x97\xe0\xae\x85@\xf0\x9d\x93\xb5\xd5\xaa\xf0\x9f\x98\x89\xf0\x9f\x98\xab\xe0\xb2\x9c\xe0\xa4\xb5h|\xe2\x94\x8f\xf0\x9f\x98\xa0\xf0\x9f\x87\xac\xe0\xae\x8f\xe2\x80\x8c\xc4\

In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [33]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


Prediction shape:  (64, 100, 500)  # (batch_size, sequence_length, vocab_size)
Mean loss:         6.214626


In [34]:
tf.exp(mean_loss).numpy()


500.00885

In [35]:
model.compile(optimizer='adam', loss=loss)


In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [37]:
EPOCHS = 50


In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/50
707/707 [==============================] - 44s 58ms/step - loss: 1.9590
Epoch 2/50
707/707 [==============================] - 45s 63ms/step - loss: 1.2896
Epoch 3/50
707/707 [==============================] - 46s 64ms/step - loss: 1.1569
Epoch 4/50
707/707 [==============================] - 45s 63ms/step - loss: 1.0855
Epoch 5/50
707/707 [==============================] - 45s 63ms/step - loss: 1.0323
Epoch 6/50
707/707 [==============================] - 46s 63ms/step - loss: 0.9893
Epoch 7/50
707/707 [==============================] - 47s 65ms/step - loss: 0.9529
Epoch 8/50
707/707 [==============================] - 45s 62ms/step - loss: 0.9223
Epoch 9/50
707/707 [==============================] - 47s 65ms/step - loss: 0.8959
Epoch 10/50
707/707 [==============================] - 45s 63ms/step - loss: 0.8748
Epoch 11/50
707/707 [==============================] - 46s 64ms/step - loss: 0.8567
Epoch 12/50
707/707 [==============================] - 47s 65ms/step - loss: 0.8431
E

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states


In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [41]:
start = time.time()
states = None
next_char = tf.constant(['Semdguru:'])
result = [next_char]

for n in range(280):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


Semdguru: ofcumedt; stim you cous now.
- jochas hoos in indeming ic you live who make is ent do and stopat you have faniinuru to you nouls? actis beling "if youlet not somet of i agded forr losable sing pets doan ets, you good, inda by is life fory a giviond if your liss th it, thible don 

________________________________________________________________________________

Run time: 1.2136638164520264


In [42]:
tf.saved_model.save(one_step_model, '/content/drive/MyDrive/model/one_step')
one_step_reloaded = tf.saved_model.load('/content/drive/MyDrive/model/one_step')



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step/assets


In [54]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))


ROMEO:
- all ion what you nes wirt comiliveyrenjodse ofs. if succlusseseress you, of sommo to chan. doent.


In [44]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}


In [45]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [46]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [47]:
model.fit(dataset, epochs=1)


707/707 [==============================] - 48s 64ms/step - loss: 1.9705


In [48]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))


Epoch 1 Batch 0 Loss 1.6985
Epoch 1 Batch 50 Loss 1.4390
Epoch 1 Batch 100 Loss 1.3204
Epoch 1 Batch 150 Loss 1.2474
Epoch 1 Batch 200 Loss 1.2880
Epoch 1 Batch 250 Loss 1.2404
Epoch 1 Batch 300 Loss 1.2077
Epoch 1 Batch 350 Loss 1.1883
Epoch 1 Batch 400 Loss 1.1355
Epoch 1 Batch 450 Loss 1.1359
Epoch 1 Batch 500 Loss 1.1463
Epoch 1 Batch 550 Loss 1.3472
Epoch 1 Batch 600 Loss 1.2241
Epoch 1 Batch 650 Loss 1.1896
Epoch 1 Batch 700 Loss 1.2614

Epoch 1 Loss: 1.2595
Time taken for 1 epoch 44.78 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.4616
Epoch 2 Batch 50 Loss 1.3450
Epoch 2 Batch 100 Loss 1.2225
Epoch 2 Batch 150 Loss 1.2594
Epoch 2 Batch 200 Loss 1.1303
Epoch 2 Batch 250 Loss 1.1581
Epoch 2 Batch 300 Loss 1.1136
Epoch 2 Batch 350 Loss 1.1140
Epoch 2 Batch 400 Loss 0.9946
Epoch 2 Batch 450 Loss 0.9734
Epoch 2 Batch 500 Loss 0.9953
Epoch 2 Batch 550 Loss 1.1030
Epoch 2 Batch 600 Loss 1.0971
Epoch 2 Batch 650 Loss 1.1644


In [49]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [50]:
start = time.time()
states = None
next_char = tf.constant(['BotTweet:'])
result = [next_char]

for n in range(250):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

BotTweet:

• free that you're sad you're putting out the way you are meant and talk, list a different, what we think, however ♥different of us are haddayou'll make you happy if you thank you for it.  the more you learn never talking about the ground with the  

________________________________________________________________________________

Run time: 1.2185277938842773


In [55]:
tf.saved_model.save(one_step_model, '/content/drive/MyDrive/model/one_step2')
#one_step_reloaded = tf.saved_model.load('/content/drive/MyDrive/model/one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step2/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step2/assets


Status(_api=<tweepy.api.API object at 0x7f1c434e68d0>, _json={'created_at': 'Sat Jul 17 19:11:55 +0000 2021', 'id': 1416475773364367360, 'id_str': '1416475773364367360', 'text': "BotTweet:\n\n• free that you're sad you're putting out the way you are meant and talk, list a different, what we thin… https://t.co/nJbdYABxXB", 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/nJbdYABxXB', 'expanded_url': 'https://twitter.com/i/web/status/1416475773364367360', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">SpiritTweeter</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1415908432452874243, 'id_str': '1415908432452874243', 'name': 'Roman', 'screen_name': 'RoBotMan_001', '